In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandas_datareader import data as web
import warnings
warnings.filterwarnings("ignore")

prices = pd.DataFrame()
tickers = ['ITUB3.SA', 'BBDC3.SA', 'BBAS3.SA', 'SANB3.SA', '^BVSP']
for i in tickers:
    prices[i] = web.get_data_yahoo(i,'01/01/2008')['Adj Close']

In [ ]:
prices.tail()

In [ ]:
prices.rename(columns ={'ITUB3.SA':'ITUB', 'BBDC3.SA':'BBDC','BBAS3.SA':'BBAS','SANB3.SA':'SANB', '^BVSP':'IBOV'},inplace = True)
prices['IBOV'] = prices['IBOV']/1000
prices.reset_index(inplace = True)

In [ ]:
prices.dropna(subset = ['IBOV'], inplace = True)
prices.IBOV.isnull().sum()

In [ ]:
tickers = list(prices.drop(['Date'], axis = 1).columns)
plt.figure(figsize=(16,6))

for i in tickers:
        plt.plot(prices['Date'], prices[i])
plt.legend(tickers)
plt.grid()
plt.title("Cotação x tempo", fontsize = 25)
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(prices['Date'],prices['ITUB'].rolling(window = 90).mean())
plt.plot(prices['Date'], prices['ITUB'], alpha = 0.8)
plt.plot(prices['Date'],prices['ITUB'].rolling(window = 365).mean())
plt.grid()
plt.title('Cotações diárias e médias móveis de ITUB3', fontsize = 15)
plt.legend(['Média móvel trimestral','Cotação diária','Média móvel anual'])
plt.show()

In [ ]:
import csv
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup

url = 'https://www.fundsexplorer.com.br/ranking'

# Data Cleansing
# 'R$' => ''
# '%' => ''
# '.0' => ''
# '.' => ''
# ',' => '.'
# 'N/A' => ''

print("Starting...{}".format(datetime.now()))

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

data = []
table = soup.find(id="table-ranking")

table_head = table.find('thead')

rows = table_head.find_all('tr')
for row in rows:
    cols = row.find_all('th')
    colsd = [ele.get_text(separator=" ").strip() for ele in cols]
    data.append([ele for ele in colsd])

table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    colsd = [ele.text.replace('R$','').replace('%','').replace('.0','').replace('.','').replace('N/A','').replace(',','.').strip() for ele in cols]
    data.append([ele for ele in colsd])

file = open("fii.csv", "w")

wtr = csv.writer(file, delimiter=';', lineterminator='\n')
for x in data : wtr.writerow(x)

file.close()

print("Finish...{}".format(datetime.now()))

time.sleep(1)

In [20]:
from tkinter import Y
import pandas as pd
import numpy as np
import matplotlib as plt
import math

df = pd.read_csv("./fii.csv", sep=";")
# df.set_index("Código do fundo").loc["VGHF11"]

df["Magic Number"] = df["Preço Atual"]/df["Dividendo"]
df["Magic Number"] = df["Magic Number"].apply(np.ceil)
df = df.sort_values(by="Magic Number", ascending=False)

df[["Código do fundo", "Magic Number"]].value_counts

<bound method DataFrame.value_counts of     Código do fundo  Magic Number
0            FIVN11           inf
23           CJCT11           inf
17           RBDS11           inf
16           KNRE11           inf
31           CARE11           inf
..              ...           ...
104          SPVJ11           NaN
158          STRX11           NaN
177          BTSG11           NaN
212          BBIM11           NaN
214         NVIF11B           NaN

[284 rows x 2 columns]>